# Testing different embedding schemes for information retrieval
## Step 1: Load sample text

In [1]:
import numpy as np
import pandas as pd


/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def read_txt(path):
    with open(path, 'r', encoding="utf-8") as f:
        text = f.readlines()
#         text = [x.decode("utf-8") for x in f.readlines()]
    return text

text = read_txt('./data/fund_guide_extend.txt')
# text = [x.decode("utf-8") for x in text]

In [3]:
condition_terms = []
stringg=''
for tex in text:
    if (tex=='\n'):
        if (stringg != ''):
            condition_terms.append(stringg)
            stringg=''
        else: pass
    else: stringg+=tex
# condition_terms=[x.replace('\n', '. ') for x in condition_terms]

In [4]:
print(len(condition_terms))
condition_terms[2]

148


'OVERVIEW\nCompliance with Code of Ethics, Research Integrity and Responsible Research Conduct\n3. The Host Institution, Director of Research, each Lead Principal Investigator, research personnel and all other persons involved in the Research shall in carrying out the Research comply with:\na. The provisions of any relevant laws of the Republic of Singapore, statutes, regulations, by-laws, rules, guidelines and requirements applicable to it as the same may be amended or varied from time to time; and\nb. All applicable policies and procedures adopted by NRF as the same may be amended or varied from time to time.\n4. The Lead Principal Investigator of a project has the responsibility to conduct research in compliance with the Host Institution’s code of ethics, research integrity and responsible research conduct. \n5. The Host Institution also has responsibility to ensure researchers’ compliance with best practices as well as ethical, legal and professional standards relevant to their res

# Step 2: Load sample questions

In [5]:
import pandas as pd
df_queries = pd.read_csv('./data/Consolidated emails.csv', encoding='iso-8859-1')
print(len(df_queries))

73


In [6]:
# Answer collection
initial_list=df_queries['Email Queries'].values
modified_list=[val for val in initial_list for _ in (0, 1, 2)]
df_comparisons = pd.DataFrame({'queries' : modified_list})

# Step 3: define similarity function
The cosine similarity function returns the cosine similarity given a query string, an encoder, and an array of knowledgebase embeddings

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
def cosine_sim_results(query_str, encoder, kb_embeddings, **kwargs):
    if kwargs:
        qn_embedding = encoder(query_str, kwargs.get('tokenize', None))
    else:
        qn_embedding = encoder(query_str)
    results = cosine_similarity(kb_embeddings, qn_embedding)
    return results

# Utility functions

In [9]:
def dummy_embed_fn(vector):
    return vector

# Exp 1: Test InferSent model

In [10]:
from InferSent.models import InferSent
import torch
V = 1
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [11]:
W2V_PATH = 'fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [12]:
infersent.build_vocab(condition_terms, tokenize=True)

Found 2646(/2721) words with w2v vectors
Vocab size : 2646


In [13]:
infersent.update_vocab(condition_terms, tokenize=True)

Found 0(/75) words with w2v vectors
New vocab size : 2646 (added 0 words)


In [14]:
response_results = infersent.encode(condition_terms, tokenize=True)
questions = list(df_queries['Email Queries'])
question_results = infersent.encode(questions, tokenize=True)

In [15]:
print(response_results.shape, question_results.shape)

(148, 4096) (73, 4096)


In [16]:
# sample question
# question=['how frequent can i get disbursements?']
# condition_terms[cosine_sim_results(question, infersent.encode, embeddings, tokenize=True).argmax()]

In [17]:
# question=['who is RESPONSIBLE FOR DATA CHARGES?']
# sortargs=cosine_sim_results(question, infersent.encode, embeddings, tokenize=True).argsort(axis=0)
# print(sortargs.shape)
# for ii,arg in enumerate(sortargs[::-1,0]):
#     print(ii, condition_terms[arg])
#     if ii==4:
#         break

In [18]:
# list(cosine_sim_results(question, infersent.encode, embeddings, tokenize=True).argsort(axis=0).shape[-2:])

## Test with example queries

In [19]:
responses=[]
for index, ii in enumerate(df_queries.iterrows()):
    print('QN: ', ii[1]['Email Queries'])
    sortargs=np.flip(cosine_sim_results(question_results[index].reshape(1, -1), dummy_embed_fn, response_results).argsort(axis=0))
    for ans in range(3):
        responses.append(condition_terms[sortargs[ans,0]])
        print('ANS: ', condition_terms[sortargs[ans,0]])
df_comparisons['infersent']=responses

QN:  SMRT-NTU Corp Lab would like to get NRF approval on the following matter. 
We would like to add  Project endorsed by LTA  on our 13 posters in the corp lab . Is NRF ok with this request ? 
The reason for the adding of this words is that SMRT wants to be more aligned with LTA for their support .
 
If its ok with NRF , I will proceed to prepare it for the opening ceremony .

ANS:  POST-AWARD MANAGEMENT
Progress Reports and Final Report of a FI
14. Programme Offices are to submit an annual progress report to NRF for each FI using the prescribed format in Annex J. A consolidated overall progress report for all FIs under its charge should be submitted as well.
15. The annual progress reports shall be submitted within 3 months after the end of each Financial Year. NRF reserves the right to demand for more frequent progress reports, if necessary.
16. The submission of annual progress reports is to continue until the end of the last project within a FI, following which, a final report m


ANS:  Collaboration Guidelines
3. Where possible, the Collaborators’ staff should be resident in Singapore, or be re-located to Singapore to undertake the research, although it is recognised that this may not always be possible in the case of Collaborators based overseas. In particular, it is understood that where the Research (and consequently, the Funding) relate to a joint grant call with an overseas funding agency or organisation, the Collaborators will be based overseas and the Collaborators’ scope of work under the Research will be undertaken overseas.

ANS:  Collaboration Guidelines
2. The work in connection with the Research performed pursuant to the collaboration with the Collaborators should, to the extent possible, be carried out in Singapore. The Institutions are not permitted to contract out the whole or a substantial part of the Research to Collaborators.

ANS:  Collaboration Guidelines
5. Collaborators accessing and using Assets acquired using the Funding may only do so

ANS:  Collaboration Guidelines
4. The Collaborators are not permitted to receive, directly or indirectly, any part of the Funding, whether in cash or in the form of Assets acquired using the Funding or otherwise. All Assets acquired using the Funding must be located in Singapore and maintained within the control of the Institutions.

QN:  We would like to inform that we are disposing a faulty KVM procured under the initial data centre setup, funded through Fujitsu cash contribution.  The disposal will be carried out in accordance to SMUs disposal guideline.
ANS:  Collaboration Guidelines
4. The Collaborators are not permitted to receive, directly or indirectly, any part of the Funding, whether in cash or in the form of Assets acquired using the Funding or otherwise. All Assets acquired using the Funding must be located in Singapore and maintained within the control of the Institutions.

ANS:  Collaboration Guidelines
3. Where possible, the Collaborators’ staff should be resident in Si

In [20]:
# responses=[]
# for ii in df_queries.iterrows():
#     print('QN: ', ii[1]['Email Queries'])
#     sortargs=np.flip(cosine_sim_results([ii[1]['Email Queries']], infersent.encode, embeddings, tokenize=True).argsort(axis=0))
#     for ans in range(3):
#         print(sortargs[ans, 0])
#         responses.append(condition_terms[sortargs[ans,0]])
#         print('ANS: ', condition_terms[sortargs[ans,0]])
# #     answer = condition_terms[cosine_sim_results([ii[1]['Email Queries']], infersent.encode, embeddings, tokenize=True).argmax()]
# #     print('ANS: ', answer)
# #     responses.append(answer)
# #     print('\n')
# df_comparisons['infersent']=responses

# Exp 2: Google universal sentence encoder

In [21]:
# !pip install tensorflow-gpu
# !pip install tensorflow-hub
import tensorflow as tf
import tensorflow_hub as hub

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [22]:
#download the model to local so it can be used again and again
# !mkdir google_use
# Download the module, and uncompress it to the destination folder. 
# !curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC ./google_use

In [23]:
embed = hub.Module("./google_use")

Instructions for updating:
Colocations handled automatically by placer.


W0913 07:56:00.905652 140286982317824 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [24]:
def use_embed(terms):
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(terms))
    return message_embeddings

# to only load session once.
# def embed_useT(module):
#     with tf.Graph().as_default():
#         sentences = tf.placeholder(tf.string)
#         embed = hub.Module(module)
#         embeddings = embed(sentences)
#         session = tf.train.MonitoredSession()
#     return lambda x: session.run(embeddings, {sentences: x})


In [25]:
response_results = use_embed(condition_terms)
all_questions = df_queries['Email Queries']
questions=list(all_questions)
question_results = use_embed(questions)
print(response_results.shape, question_results.shape)

InternalError: Dst tensor is not initialized.
	 [[node checkpoint_initializer_20 (defined at /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow_hub/native_module.py:407) ]]

Caused by op 'checkpoint_initializer_20', defined at:
  File "/anaconda/envs/py35/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/envs/py35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/anaconda/envs/py35/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/anaconda/envs/py35/lib/python3.5/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "/anaconda/

In [ ]:
# responses=[]
# for ii in df_queries.iterrows():
#     print('QN: ', ii[1]['Email Queries'])
#     answer = condition_terms[cosine_sim_results([ii[1]['Email Queries']], use_embed, message_embeddings).argmax()]
#     print('ANS: ', answer)
#     responses.append(answer)
#     print('\n')
# df_comparisons['use']=responses

In [ ]:

responses=[]
for index, ii in enumerate(df_queries.iterrows()):
    print('QN: ', ii[1]['Email Queries'])
    sortargs=np.flip(cosine_sim_results(question_results[index].reshape(1, -1), dummy_embed_fn, response_results).argsort(axis=0))
    for ans in range(3):
        responses.append(condition_terms[sortargs[ans,0]])
        print('ANS: ', condition_terms[sortargs[ans,0]])
df_comparisons['use']=responses

In [ ]:
# responses=[]
# for ii in df_queries.iterrows():
#     print('QN: ', ii[1]['Email Queries'])
#     sortargs=np.flip(cosine_sim_results([ii[1]['Email Queries']], use_embed, message_embeddings).argsort(axis=0))
#     for ans in range(3):
#         responses.append(condition_terms[sortargs[ans,0]])
#         print('ANS: ', condition_terms[sortargs[ans,0]])
# df_comparisons['use']=responses

# Exp 3: Test the new QnA USE

In [ ]:
# !pip install sentencepiece
!pip install tf-sentencepiece

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

In [ ]:
def init_model():
    # Set up graph.
    g = tf.Graph()
    with g.as_default():
        embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/1")
        # put placeholders
        question = tf.placeholder(dtype=tf.string, shape=[None])  # question
        response = tf.placeholder(dtype=tf.string, shape=[None])  # response
        response_context = tf.placeholder(dtype=tf.string, shape=[None])  # response context
        
        question_embeddings = embed(
        dict(input=question),
        signature="question_encoder", as_dict=True)

        response_embeddings = embed(
        dict(input=response,
             context=response_context),
        signature="response_encoder", as_dict=True)

        init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
    g.finalize()

    # Initialize session.
    session = tf.Session(graph=g, config=tf.ConfigProto(log_device_placement=True))
    session.run(init_op)
    return session, question_embeddings, response_embeddings, question, response, response_context

In [ ]:
# define inputs
responses = condition_terms
response_contexts = responses # no need to provide context
all_questions = df_queries['Email Queries']
questions=list(all_questions)

In [ ]:
session, question_embeddings, response_embeddings = init_model(questions, responses, response_contexts)

In [ ]:
# Compute embeddings.
response_results = session.run(response_embeddings)


In [ ]:
question_results = session.run(question_embeddings)
session.close()

In [ ]:
response_results['outputs'].shape

In [ ]:
# responses = []
# for index, ii in enumerate(df_queries.iterrows()):
#     print('QN: ', ii[1]['Email Queries'])
#     answer=condition_terms[cosine_sim_results(question_results['outputs'][index].reshape(1, -1), dummy_embed_fn, response_results['outputs']).argmax()]
#     print('ANS: ', answer)
#     responses.append(answer)
#     print('\n')
# df_comparisons['use_qa']=responses

In [ ]:
responses=[]
for index, ii in enumerate(df_queries.iterrows()):
    print('QN: ', ii[1]['Email Queries'])
    sortargs=np.flip(cosine_sim_results(question_results['outputs'][index].reshape(1, -1), dummy_embed_fn, response_results['outputs']).argsort(axis=0))
    for ans in range(3):
        responses.append(condition_terms[sortargs[ans,0]])
        print('ANS: ', condition_terms[sortargs[ans,0]])
df_comparisons['use_qa']=responses

# Save to csv

In [ ]:
df_comparisons.to_csv('./predictions_3_extend_batch2.csv', index=False)

# Test insurance db

In [ ]:
from pathlib import Path
datapath=Path('./data')
df_query = pd.read_csv(datapath/'insuranceQA/V2/InsuranceQA.question.anslabel.raw.100.pool.solr.test.encoded', delimiter='\t', header=None)
df_doc = pd.read_csv(datapath/'insuranceQA/V2/InsuranceQA.label2answer.raw.encoded', delimiter='\t', header=None)
df_ind2word = pd.read_csv(datapath/'insuranceQA/V2/vocabulary', sep='\t', header=None, quotechar='', quoting=3, keep_default_na=False)
dict_ind2word = pd.Series(df_ind2word[1].values,index=df_ind2word[0].values).to_dict()
# dict_ind2word.update(dict_ind2word_v1)

## Extract only questions that have answers
The dataset has this weird thing where the questions that have no correct answers have random answers in the answer column that does not match the question.

In [ ]:
def question_cleaner(df_query):
    kb=([int(xx) for xx in (df_query[3].iloc[0]).split(' ')])
    gt = [int(xx) for xx in (df_query[2].iloc[0]).split(' ')]
    ct=0
    negg=0
    withans=[]
    for ii in range(len(df_query)):
        kb=[int(xx) for xx in (df_query[3].iloc[ii]).split(' ')]
        gt = [int(xx) for xx in (df_query[2].iloc[ii]).split(' ')]
        if bool(set(gt) & set(kb)):
            withans.append(ii)
        else:
            negg+=1
    print('total:{}, removed:{}, remainder:{}'.format(len(df_query), negg, len(withans)))
    return df_query.iloc[withans]

In [ ]:
df_query=question_cleaner(df_query)
df_doc=df_doc.set_index(0)

In [ ]:
def wordifier(tokes):
    return ' '.join([dict_ind2word[ind] for ind in tokes.strip().split(' ')])
df_doc['text']=df_doc.apply(lambda x: wordifier(x[1]), axis=1)
df_query['text']=df_query.apply(lambda x: wordifier(x[1]), axis=1)
display(df_query.head(1))
display(df_doc.head(1))

## Print example question and answers

In [ ]:
def display_qn_and_ans(df_query, index=0):
    kb=[int(xx) for xx in (df_query[3].iloc[index]).split(' ')]
    gt = [int(xx) for xx in (df_query[2].iloc[index]).split(' ')]
    print('Question is: {}'.format(df_query['text'].iloc[index]))
    print('Answer index: ', gt)
    print('Answers: ', df_doc.loc[gt, 'text'].values)
display_qn_and_ans(df_query, index=2)

## Vectorize all the questions
Start by initializing the model

In [ ]:
session, question_embeddings, response_embeddings, question, response, response_context = init_model()

## Compute question embeddings

In [ ]:
question_results = session.run(question_embeddings, feed_dict={question:df_query['text'].tolist()})

In [ ]:
question_results['outputs'].shape

In [ ]:
score=0
total=0
predictions=[]
for ii, question_vector in enumerate(question_results['outputs']):
    kb=[int(xx) for xx in (df_query[3].iloc[ii]).split(' ')]
    gt = [int(xx) for xx in (df_query[2].iloc[ii]).split(' ')]
    doc_vectors = session.run(response_embeddings, feed_dict={
        response:df_doc.loc[kb]['text'].tolist(),
        response_context:df_doc.loc[kb]['text'].tolist()
    })
    cossim = cosine_similarity(doc_vectors['outputs'], question_vector.reshape(1, -1))
    sortargs=np.flip(cossim.argsort(axis=0))
    returnedans = [kb[jj[0]] for jj in sortargs[:3]]
    predictions.append(returnedans)
    if bool(set(gt) & set(returnedans)):
        score+=1
    total+=1
    

In [ ]:
print(score/total)

In [ ]:
predictions

In [ ]:

# question_results = session.run(question_embeddings)
response_results = session.run(response_embeddings, 
                               feed_dict={
          response: df_doc['text'].iloc[:10].tolist(),
          response_context: df_doc['text'].iloc[:10].tolist()
      })
# session.close()

In [ ]:
df_qwa=df_query.iloc[withans]
new_kb=[int(xx) for xx in (df_qwa[3].iloc[0]).split(' ')]
new_gt = [int(xx) for xx in (df_qwa[2].iloc[0]).split(' ')]

In [ ]:
import numpy as np
chunk=200
response_results=np.array([])
df_vect_results=pd.DataFrame()
for ii in range(0, len(df_doc), chunk):
    print(ii)
    vector_batch=session.run(response_embeddings, feed_dict={response:df_doc['text'].iloc[ii:ii+chunk].tolist(),
                                                                 response_context:df_doc['text'].iloc[ii:ii+chunk].tolist()})
    df_vectors = pd.DataFrame(vector_batch['outputs'], dtype=np.float32)
    df_vect_results = pd.concat([df_vect_results, df_vectors], axis=0)
    
    

In [ ]:
# df_doc['vectors'] = response_results
df_vectors=pd.DataFrame(question_results['outputs'], dtype=np.float32)
df_query = pd.concat([df_query, df_vectors], axis=1)
# df_doc.to_csv(datapath/'insurance_doc.csv', index=False)
df_query.to_csv(datapath/'insurance_query.csv', index=False)

In [ ]:
df_vectors